In [7]:
# import modules
import numpy as np
from scipy.optimize import root

In [27]:
PH2O = 12.63  # bar
R = 0.083145 # L bar /K /mol
T = 190 + 273.15 #K
Mi = 23/18.02 #mol
Vt = 145 #mL

def objective_function(init_VG):
    VG_L = init_VG / 1000 # L
    MG = PH2O * VG_L / R/T # mol
    ML = Mi - MG # mol
    VL_L = ML * 18.02

    return VG_L*1000 + VL_L - Vt

init_VG = Vt/10
result = root(objective_function, init_VG)


In [28]:
result.x[0]/1000


np.float64(0.12272532815069112)

In [6]:
import numpy as np
from scipy.optimize import root

def calculate_equilibrium_scipy(temp_C, total_pressure_bar, total_volume_mL, initial_water_mL):
    """
    scipy.optimize를 사용하여 물과 수소가 공존하는 닫힌 계에서 열역학적 평형 상태를 계산합니다.
    
    Parameters:
    -----------
    temp_C : float
        온도 (섭씨)
    total_pressure_bar : float
        계의 총 압력 (bar)
    total_volume_mL : float
        반응기의 총 부피 (mL)
    initial_water_mL : float
        초기 물의 부피 (mL)
    
    Returns:
    --------
    dict
        평형 상태에서의 각종 값들 (기체 부피, 액체 부피, 몰수 등)
    """
    # 상수 정의
    R = 0.083145  # 기체 상수 (bar·L/mol·K)
    water_molecular_weight = 18.02  # 물의 분자량 (g/mol)
    
    # 온도 변환 (섭씨 -> 켈빈)
    temp_K = temp_C + 273.15
    
    # 200°C에서 물의 증기압 (bar)
    water_vapor_pressure = 15.55
    
    # 200°C에서 물의 밀도 (g/cm³)
    water_density = 0.865
    
    # 초기 물의 몰수 계산
    initial_water_moles = (initial_water_mL * water_density) / water_molecular_weight
    
    # 수소의 분압 계산
    hydrogen_partial_pressure = total_pressure_bar - water_vapor_pressure
    
    # 목적 함수 정의: 기체 부피를 찾기 위한 함수
    def objective_function(gas_volume_mL):
        # 기체 부피를 리터 단위로 변환
        gas_volume_L = gas_volume_mL / 1000
        
        # 기체상의 물과 수소의 몰수 계산
        gas_water_moles = (water_vapor_pressure * gas_volume_L) / (R * temp_K)
        
        # 액체상의 물의 몰수 계산
        liquid_water_moles = initial_water_moles - gas_water_moles
        
        # 액체상의 물의 부피 계산
        liquid_water_volume_mL = (liquid_water_moles * water_molecular_weight) / water_density
        
        # 기체와 액체 부피의 합이 전체 부피와 같아야 함
        return gas_volume_mL + liquid_water_volume_mL - total_volume_mL
    
    # 초기 추측값: 기체가 전체 부피의 절반을 차지한다고 가정
    initial_guess = total_volume_mL / 2
    
    # scipy.optimize.root를 사용하여 해 구하기
    result = root(objective_function, initial_guess)
    
    # 최적의 기체 부피
    gas_volume_mL = result.x[0]
    gas_volume_L = gas_volume_mL / 1000
    
    # 기체상의 물과 수소의 몰수 계산
    gas_water_moles = (water_vapor_pressure * gas_volume_L) / (R * temp_K)
    hydrogen_moles = (hydrogen_partial_pressure * gas_volume_L) / (R * temp_K)
    
    # 액체상의 물의 몰수 계산
    liquid_water_moles = initial_water_moles - gas_water_moles
    
    # 액체상의 물의 부피 계산
    liquid_water_volume_mL = (liquid_water_moles * water_molecular_weight) / water_density
    
    # 최종 결과
    final_result = {
        "temperature_C": temp_C,
        "total_pressure_bar": total_pressure_bar,
        "water_vapor_pressure_bar": water_vapor_pressure,
        "hydrogen_partial_pressure_bar": hydrogen_partial_pressure,
        "total_volume_mL": total_volume_mL,
        "gas_volume_mL": gas_volume_mL,
        "liquid_water_volume_mL": liquid_water_volume_mL,
        "gas_water_moles": gas_water_moles,
        "hydrogen_moles": hydrogen_moles,
        "liquid_water_moles": liquid_water_moles,
        "initial_water_moles": initial_water_moles,
        "convergence_success": result.success,
        "function_evaluations": result.nfev,
        "scipy_message": result.message
    }
    
    return final_result

# 문제에 주어진 조건으로 계산
result = calculate_equilibrium_scipy(
    temp_C=200,
    total_pressure_bar=50,
    total_volume_mL=145,
    initial_water_mL=23
)

# 결과 출력
print("=== 최종 결과 (SciPy 사용) ===")
print(f"온도: {result['temperature_C']}°C ({result['temperature_C'] + 273.15}K)")
print(f"총 압력: {result['total_pressure_bar']} bar")
print(f"물의 증기압: {result['water_vapor_pressure_bar']} bar")
print(f"수소의 분압: {result['hydrogen_partial_pressure_bar']} bar")
print(f"반응기 총 부피: {result['total_volume_mL']} mL")
print(f"기체 부피: {result['gas_volume_mL']:.2f} mL")
print(f"액체 물 부피: {result['liquid_water_volume_mL']:.2f} mL")
print(f"기체상 물 몰수: {result['gas_water_moles']:.6f} mol")
print(f"기체상 수소 몰수: {result['hydrogen_moles']:.6f} mol")
print(f"액체상 물 몰수: {result['liquid_water_moles']:.6f} mol")
print(f"초기 물 몰수: {result['initial_water_moles']:.6f} mol")
print(f"수렴 성공 여부: {result['convergence_success']}")
print(f"함수 호출 횟수: {result['function_evaluations']}")
print(f"SciPy 메시지: {result['scipy_message']}")

# 이전 방법과 비교를 위해 Manual Iteration 방법도 구현
def calculate_equilibrium_manual(temp_C, total_pressure_bar, total_volume_mL, initial_water_mL, tolerance=1e-6, max_iterations=100):
    # 상수 정의
    R = 0.083145  # 기체 상수 (bar·L/mol·K)
    water_molecular_weight = 18.02  # 물의 분자량 (g/mol)
    
    # 온도 변환 (섭씨 -> 켈빈)
    temp_K = temp_C + 273.15
    
    # 200°C에서 물의 증기압 (bar)
    water_vapor_pressure = 15.55
    
    # 200°C에서 물의 밀도 (g/cm³)
    water_density = 0.865
    
    # 초기 물의 몰수 계산
    initial_water_moles = (initial_water_mL * water_density) / water_molecular_weight
    
    # 수소의 분압 계산
    hydrogen_partial_pressure = total_pressure_bar - water_vapor_pressure
    
    # 반복 계산 초기화
    gas_volume_mL = total_volume_mL  # 초기 가정: 기체가 전체 부피를 차지
    liquid_water_volume_mL = 0
    
    iterations = 0
    change = float('inf')
    
    while change > tolerance and iterations < max_iterations:
        # 기체 부피를 리터 단위로 변환
        gas_volume_L = gas_volume_mL / 1000
        
        # 기체상의 물과 수소의 몰수 계산
        gas_water_moles = (water_vapor_pressure * gas_volume_L) / (R * temp_K)
        hydrogen_moles = (hydrogen_partial_pressure * gas_volume_L) / (R * temp_K)
        
        # 액체상의 물의 몰수 계산
        liquid_water_moles = initial_water_moles - gas_water_moles
        
        # 액체상의 물의 부피 계산
        new_liquid_water_volume_mL = (liquid_water_moles * water_molecular_weight) / water_density
        
        # 기체가 차지하는 새로운 부피 계산
        new_gas_volume_mL = total_volume_mL - new_liquid_water_volume_mL
        
        # 변화량 계산
        change = abs(new_gas_volume_mL - gas_volume_mL)
        
        # 값 업데이트
        gas_volume_mL = new_gas_volume_mL
        liquid_water_volume_mL = new_liquid_water_volume_mL
        
        iterations += 1
    
    # 최종 결과 (간소화된 출력)
    return {
        "gas_volume_mL": gas_volume_mL,
        "liquid_water_volume_mL": liquid_water_volume_mL,
        "gas_water_moles": gas_water_moles,
        "hydrogen_moles": hydrogen_moles,
        "liquid_water_moles": liquid_water_moles,
        "iterations": iterations,
        "converged": change <= tolerance
    }

# 두 방법 비교
manual_result = calculate_equilibrium_manual(
    temp_C=200,
    total_pressure_bar=50,
    total_volume_mL=145,
    initial_water_mL=40
)

print("\n=== 방법 비교 ===")
print("SciPy 방법:")
print(f"  기체 부피: {result['gas_volume_mL']:.6f} mL")
print(f"  액체 물 부피: {result['liquid_water_volume_mL']:.6f} mL")
print(f"  기체상 물 몰수: {result['gas_water_moles']:.6f} mol")
print(f"  기체상 수소 몰수: {result['hydrogen_moles']:.6f} mol")
print(f"  함수 호출 횟수: {result['function_evaluations']}")

print("\n수동 반복 방법:")
print(f"  기체 부피: {manual_result['gas_volume_mL']:.6f} mL")
print(f"  액체 물 부피: {manual_result['liquid_water_volume_mL']:.6f} mL")
print(f"  기체상 물 몰수: {manual_result['gas_water_moles']:.6f} mol")
print(f"  기체상 수소 몰수: {manual_result['hydrogen_moles']:.6f} mol")
print(f"  반복 횟수: {manual_result['iterations']}")

=== 최종 결과 (SciPy 사용) ===
온도: 200°C (473.15K)
총 압력: 50 bar
물의 증기압: 15.55 bar
수소의 분압: 34.45 bar
반응기 총 부피: 145 mL
기체 부피: 123.01 mL
액체 물 부피: 21.99 mL
기체상 물 몰수: 0.048624 mol
기체상 수소 몰수: 0.107722 mol
액체상 물 몰수: 1.055428 mol
초기 물 몰수: 1.104051 mol
수렴 성공 여부: True
함수 호출 횟수: 6
SciPy 메시지: The solution converged.

=== 방법 비교 ===
SciPy 방법:
  기체 부피: 123.012943 mL
  액체 물 부피: 21.987057 mL
  기체상 물 몰수: 0.048624 mol
  기체상 수소 몰수: 0.107722 mol
  함수 호출 횟수: 6

수동 반복 방법:
  기체 부피: 105.871795 mL
  액체 물 부피: 39.128205 mL
  기체상 물 몰수: 0.041848 mol
  기체상 수소 몰수: 0.092712 mol
  반복 횟수: 5
